In [2]:
import pandas as pd

#import tweepy

import config

from bots.twitter import TwitterBot
from helpers.data_helpers import save_to_parquet, clean_text, clean_tweets_df, save_to_text

# Extracting Tweets

In [3]:
# initialize twitter bot
twitter_bot = TwitterBot(bearer=config.TWTR_BEARER_TOKEN,
                         api=config.TWTR_API,
                         api_secret=config.TWTR_API_SECRET,
                         access=config.TWTR_ACCESS_TOKEN,
                         access_secret=config.TWTR_ACCESS_TOKEN_SECRET)

In [4]:
apparel = [
    'context:67.839543390668673024 -is:retweet lang:en',    # context: apparel
    'context:131.1248316002317643776 -is:retweet lang:en',  # context: athletic apparel
    'context:65.1256236649253449729 -is:retweet lang:en',   # context: fashion and beauty
    'context:131.1095391406816784384 -is:retweet lang:en',  # context: shopping
    'context:67.1486758812849635329 -is:retweet lang:en',   # context: retail sales
    'context:131.1407812892909473795 -is:retweet lang:en',  # context: ecommerce
    'context:165.1407812892909473795 -is:retweet lang:en',  # context: ecommerce
]

In [5]:
cars = [
    'context:46.781974597251964928 -is:retweet lang:en',    # context: car rental/sharing
    'context:45.781972125171060736 -is:retweet lang:en',    # context: auto
    'context:46.781972125179518977 -is:retweet lang:en',    # context: auto manufacturer
    'context:45.1196845866138533888 -is:retweet lang:en',   # context: automobile brands
    'context:131.1196845866138533888 -is:retweet lang:en',  # context: automobile brands
    'context:66.1527619563302420480 -is:retweet lang:en',   # context: cars
    'context:66.847528646185070592 -is:retweet lang:en',    # context: luxury cars
    'context:66.1177213956856799232 -is:retweet lang:en',   # context: used cars
]

In [6]:
beauty = [
    'context:65.850395585941086209 -is:retweet lang:en',    # context: beauty
    'context:131.1352035407127429120 -is:retweet lang:en',  # context: beauty industry
    'context:46.781974597222604800 -is:retweet lang:en',    # context: beauty/hygiene
    'context:131.1357424577974992897 -is:retweet lang:en',  # context: makeup collaborations
    'context:131.1354133238709542912 -is:retweet lang:en',  # context: new makeup
    'context:67.1400874108556955657 -is:retweet lang:en',   # context: eye makeup
    'context:66.854761964874088448 -is:retweet lang:en',    # context: hair care
    'context:66.855107826351357952 -is:retweet lang:en',    # context: skin care
]

In [40]:
%%timeit
dfs = []
for query in cars:
    tweets = twitter_bot.get_recent_tweets(query=query, limit=5000)
    df = pd.DataFrame.from_dict(tweets)
    dfs.append(df)

TooManyRequests: 429 Too Many Requests
Too Many Requests

In [ ]:
tweets = pd.concat(dfs, ignore_index=True)
tweets = tweets.loc[~tweets['id'].duplicated()].copy()

In [ ]:
print(len(tweets))

In [10]:
save_to_parquet(data_dir=config.RAW_DATA_DIR, df=tweets, name='tweets_apparel_27k')

2023-03-30 16:01:06,703 - INFO     | helpers.data_helpers | tweets_apparel_27k.parquet saved.


# Dataset Creation

In [32]:
import glob

files = glob.glob(f'{config.RAW_DATA_DIR}/*_2023327.parquet')
files

['raw_data/tweets_apparel_32k_2023327.parquet',
 'raw_data/tweets_beauty_27k_2023327.parquet',
 'raw_data/tweets_cars_20k_2023327.parquet']

In [33]:
# combining dataframes
dfs_list = []
for file in files:
    df = pd.read_parquet(file)
    dfs_list.append(df)
dfs = pd.concat(dfs_list, ignore_index=True)

In [34]:
print(f'Total Tweets before duplicates: {len(dfs)}')
dfs = dfs.loc[~dfs['id'].duplicated()].copy()
print(f'Total Tweets after duplicates: {len(dfs)}')

Total Tweets before duplicates: 79704
Total Tweets after duplicates: 79122


In [35]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79122 entries, 0 to 79703
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          79122 non-null  int64              
 1   created_at  79122 non-null  datetime64[ns, UTC]
 2   text        79122 non-null  object             
 3   entities    79122 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 3.0+ MB


In [36]:
# converting dates
dfs['created_at'] = pd.to_datetime(dfs['created_at'].dt.strftime('%Y-%m-%d'))

In [37]:
# cleaning text
dfs['text'] = dfs['text'].map(lambda x: clean_text(x))

In [38]:
tweets_df = clean_tweets_df(df=dfs, text_col='text', n=2)
save_to_parquet(data_dir=config.DATA_DIR, df=tweets_df, name='all_tweets')
print(f'Total tweets after removing short Tweets: {len(tweets_df)}')

2023-03-30 16:16:20,173 - INFO     | helpers.data_helpers | all_tweets.parquet saved.


Total tweets after removing short Tweets: 78356


In [39]:
tweets_df.head()

,id,created_at,entities,full_text
0,1640367128854564864,2023-03-27,"[Entertainment & Leisure Business, B2B, Baseba...",\nTWEET: hahdhdh thats why he is practicing i ...
1,1640367079076376579,2023-03-27,"[B2B, Services - B2B, Technology Business, Pro...",\nTWEET: that's right - we're using #ai to hel...
2,1640366751463374853,2023-03-27,"[Hasbro, Products - B2B, B2B]",\nTWEET: indy goes trick or treating as two fa...
3,1640366569673850881,2023-03-27,"[B2B, Services - B2B, Technology Business, Dat...",\nTWEET: forget the diy headaches when you cho...
4,1640366205818003462,2023-03-27,"[Samsung , NVIDIA, B2B, AMD, Samsung USA, Sams...",\nTWEET: the global semiconductor supply chain...


### Shuffling and Slicing

In [41]:
tweets_df = pd.read_parquet(f'{config.DATA_DIR}/all_tweets_2023327.parquet')

In [42]:
tweets_df = tweets_df.sample(frac=1).reset_index(drop=True)

In [43]:
idx = int(len(tweets_df) * 0.7)
idx

54849

In [44]:
input_tweets = tweets_df[:idx]
eval_tweets = tweets_df[idx:]

In [45]:
save_to_text(df=input_tweets, col='full_text', out_dir=config.DATA_DIR, name='input_tweets_2023327')
save_to_parquet(data_dir=config.DATA_DIR, df=eval_tweets, name='eval_tweets')

2023-03-30 16:24:53,136 - INFO     | helpers.data_helpers | Saved corpus to .txt file: input_tweets_2023327.txt
2023-03-30 16:24:53,180 - INFO     | helpers.data_helpers | eval_tweets.parquet saved.
